In [2]:
# cell の大きさを100％にする
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [3]:
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta

df_customer_join = pd.read_csv("./customer_join.csv")
df_use_log_months = pd.read_csv("./use_log_months.csv")


In [17]:
df_use_log =  df_use_log_months.copy()
df_use_log["month of use"] = pd.to_datetime(df_use_log["年月"],format="%Y%m")
gp = df_use_log.groupby("customer_id")

tmp = list()
for customer_id, df in gp:
    df = df.sort_values(by="month of use")
    df["count_1"] = df["count"].shift()
    tmp.append(df)

df_use_log_data = pd.concat(tmp)
df_use_log_data = df_use_log_data.dropna()


In [20]:
df_use_log_data


,年月,customer_id,count,month of use,count_1
2991,201805,AS002855,5,2018-05-01,4.0
6054,201806,AS002855,5,2018-06-01,5.0
9158,201807,AS002855,5,2018-07-01,5.0
12291,201808,AS002855,3,2018-08-01,5.0
15424,201809,AS002855,7,2018-09-01,3.0
...,...,...,...,...,...
24646,201811,TS999855,4,2018-11-01,4.0
27759,201812,TS999855,5,2018-12-01,4.0
30851,201901,TS999855,7,2019-01-01,5.0
33888,201902,TS999855,4,2019-02-01,7.0


In [53]:
df_all_merge = df_use_log_data.merge(df_customer_join, on="customer_id", how="left")
df_all_merge["period"] = (df_all_merge["month of use"] - pd.to_datetime(df_all_merge["start_date"]))//np.timedelta64(1, 'M')
df_all_merge.head().T

,0,1,2,3,4
年月,201805,201806,201807,201808,201809
customer_id,AS002855,AS002855,AS002855,AS002855,AS002855
count,5,5,5,3,7
month of use,2018-05-01 00:00:00,2018-06-01 00:00:00,2018-07-01 00:00:00,2018-08-01 00:00:00,2018-09-01 00:00:00
count_1,4.0,5.0,5.0,5.0,3.0
name,XXXX,XXXX,XXXX,XXXX,XXXX
class,C03,C03,C03,C03,C03
gender,F,F,F,F,F
start_date,2016-11-01,2016-11-01,2016-11-01,2016-11-01,2016-11-01
end_date,NaN,NaN,NaN,NaN,NaN


In [54]:
df_exit_customer = df_all_merge[df_all_merge["is_deleted"] == 1]
df_continue_customer = df_all_merge[df_all_merge["is_deleted"] == 0]


In [55]:
df_continue_customer["exit_date"] = df_continue_customer["month of use"].apply(lambda x: x-relativedelta(months=1))


/home/shinseitaro/workspace/github/python-data-analysis-100/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [57]:
df_continue_customer = df_continue_customer.sort_values(by="month of use").groupby("customer_id").last()

In [59]:
df_exit_customer = df_exit_customer.sort_values(by="month of use").groupby("customer_id").last()
df_exit_customer

,年月,count,month of use,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,period
customer_id,,,,,,,,,,,,,,,,,,,,,
AS008805,201903,1,2019-03-01,6.0,XXXXX,C01,M,2018-06-07,2019-03-31,CA2,...,10500,入会費半額,4.000000,4.0,8,1,1,2019-03-31,9,8
AS015746,201808,3,2018-08-01,3.0,XXXXX,C01,M,2017-12-01,2018-08-31,CA3,...,10500,入会費無料,2.600000,3.0,4,1,0,2018-08-31,8,7
AS019120,201810,3,2018-10-01,5.0,XXXX,C01,M,2018-08-07,2018-10-31,CA1,...,10500,通常,5.333333,5.0,8,3,0,2018-10-31,2,1
AS025956,201902,3,2019-02-01,1.0,XXXXXX,C01,M,2018-05-09,2019-02-28,CA2,...,10500,入会費半額,4.400000,4.5,7,1,1,2019-02-28,9,8
AS029624,201901,2,2019-01-01,5.0,XXXXX,C01,M,2018-05-08,2019-01-31,CA2,...,10500,入会費半額,4.777778,5.0,8,1,1,2019-01-31,8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TS963433,201812,1,2018-12-01,4.0,XX,C02,M,2018-07-14,2018-12-31,CA1,...,7500,通常,4.166667,4.5,6,1,0,2018-12-31,5,4
TS971314,201901,4,2019-01-01,7.0,XXXX,C03,M,2018-12-01,2019-01-31,CA3,...,6000,入会費無料,5.500000,5.5,7,4,1,2019-01-31,1,1
TS973306,201807,1,2018-07-01,2.0,XX,C01,M,2017-07-01,2018-07-31,CA2,...,10500,入会費半額,2.750000,3.0,4,1,1,2018-07-31,12,11


In [61]:
len(df_continue_customer), len(df_exit_customer)

(2777, 1229)

In [67]:
df_predict_data = pd.concat([df_continue_customer, df_exit_customer])
df_predict_data = df_predict_data.reset_index()


In [68]:
df_predict_data.isna().sum()

customer_id             0
年月                      0
count                   0
month of use            0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2777
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
period                  0
exit_date            1229
dtype: int64

In [69]:
target_columns = ["campaign_name","class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
df_predict_data_updated = df_predict_data[target_columns]
df_predict_data_updated.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,ナイト,F,5.0,1,27,0
1,通常,オールタイム,F,6.0,1,39,0
2,入会費半額,オールタイム,M,4.0,1,9,0
3,通常,オールタイム,M,5.0,1,43,0
4,通常,ナイト,M,6.0,1,20,0


In [70]:
df_predict_data_updated = pd.get_dummies(df_predict_data_updated)
df_predict_data_updated.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1,27,0,0,0,1,0,0,1,1,0
1,6.0,1,39,0,0,0,1,1,0,0,1,0
2,4.0,1,9,0,1,0,0,1,0,0,0,1
3,5.0,1,43,0,0,0,1,1,0,0,0,1
4,6.0,1,20,0,0,0,1,0,0,1,0,1


In [75]:
df_predict_data_updated = df_predict_data_updated.drop(["gender_M", "campaign_name_通常", "class_name_ナイト"], axis=1)

In [76]:
df_predict_data_updated.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1,27,0,0,0,0,0,1
1,6.0,1,39,0,0,0,1,0,1
2,4.0,1,9,0,1,0,1,0,0
3,5.0,1,43,0,0,0,1,0,0
4,6.0,1,20,0,0,0,0,0,0


In [82]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exist_member = df_predict_data_updated[df_predict_data_updated["is_deleted"]==1]
continue_member = df_predict_data_updated[df_predict_data_updated["is_deleted"]==0].sample(len(exist_member))


In [85]:
X = pd.concat([exist_member, continue_member])
y = X["is_deleted"]
X = X.drop(["is_deleted"], axis=1)


In [95]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)


In [98]:
X_test

,count_1,routine_flg,period,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
3329,3.0,0,7,0,0,0,1,0
3581,2.0,0,1,0,1,0,0,0
2538,7.0,1,23,0,0,0,0,1
761,9.0,1,18,1,0,1,0,0
370,4.0,1,21,0,0,0,1,1
...,...,...,...,...,...,...,...,...
3112,2.0,1,7,0,0,0,0,0
3599,2.0,1,5,1,0,1,0,1
202,5.0,1,16,0,0,0,1,1
3694,2.0,0,11,1,0,1,0,1


In [99]:
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_predict = model.predict(X_test)
print(y_test_predict)

[1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 1
 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1
 1 0 1 1 1 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1
 1 1 0 0 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0
 1 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 0 0 0 1 1 0
 1 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 1 1
 1 0 1 0 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 1 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 0 1 0 1 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1
 0 1 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 1 0 1 1 1 0 1 0 1 0 0 1 0 1 0 0
 0 1 0 0 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 1 1 0 1
 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 0 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 1 0 0
 0 1 0 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1
 1 1 1 0 1 1 1 0 0 0 1 0 

In [100]:
df_results = pd.DataFrame({
    "y_test": y_test,
    "y_test_predict":y_test_predict,
})

df_results

,y_test,y_test_predict
3329,1,1
3581,1,1
2538,0,0
761,0,0
370,0,0
...,...,...
3112,1,1
3599,1,1
202,0,0
3694,1,1


In [106]:
(df_results["y_test"] == df_results["y_test_predict"]).value_counts()

True     580
False     35
dtype: int64

In [108]:
model.score(X_test, y_test), model.score(X_train, y_train), 

(0.943089430894309, 0.984264785675529)

In [110]:
df_importance = pd.DataFrame({
    "feature_names": X.columns,
    "coefficient": model.feature_importances_
})

df_importance

,feature_names,coefficient
0,count_1,0.339983
1,routine_flg,0.080017
2,period,0.523339
3,campaign_name_入会費半額,0.011538
4,campaign_name_入会費無料,0.010509
5,class_name_オールタイム,0.014262
6,class_name_デイタイム,0.007935
7,gender_F,0.012418


In [112]:
X_test.columns

Index(['count_1', 'routine_flg', 'period', 'campaign_name_入会費半額',
       'campaign_name_入会費無料', 'class_name_オールタイム', 'class_name_デイタイム',
       'gender_F'],
      dtype='object')

In [172]:
import random
import collections

def flatten(l):
    for el in l:
        if isinstance(el, collections.abc.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el
            

count_1 = random.choice(range(10))
routine_flg = random.choice([1,0])
period = random.choice(range(1,30))
campaign_name = random.choice(["入会費無料","入会費無料", "通常"])
class_name = random.choice(["オールタイム","デイタイム", "ナイト"])
gender = random.choice(["M","F"])

print(count_1, routine_flg, period, campaign_name, class_name, gender)

input_data = [
    count_1,
    routine_flg,
    period,
    {"入会費無料":[1,0],"入会費無料":[0,1], "通常":[0,0]}.get(campaign_name),
    {"オールタイム":[1,0],"デイタイム":[0,1], "ナイト":[0,0]}.get(class_name),
    {"M":0,"F":1}.get(gender)]

input_data =  list(flatten(input_data))
print(input_data)

print(model.predict([input_data]))
print(model.predict_proba([input_data]))


9 0 27 入会費無料 デイタイム F
[9, 0, 27, 0, 1, 0, 1, 1]
[0]
[[1. 0.]]
